# Satna proizvodnja električne energije iz solarnih elektrana

Za analizu satne proizvodnje električne energije iz solarnih elektrana korišteni su podaci dostupni na linku https://www.sunnyportal.com/Templates/PublicPagesPlantList.aspx sa filterima za BiH i snagu od 150 kWp. Sa navedenog portala se moggu preuzeti satni podaci za sve elektrane koje su u bazi, ali je preuzimanje za neke elektrane izuzetno dugotrajan zadatak. Iz tog razloga su preuzeti satni podaci za 2019., 2020. i 2021. za FNE Krenica 1 (Grude) i FNE Hodovo 1 (Stolac) i za FNE Enelectronic (Čitluk). Ove FNE imaju opciju "Plant overview" kojim se olakšava preuzimanje fajlova na satnoj osnovi, te su one odabrane za cjelokupnu analizu. Ukupna godišnja proizvodnja će se skalirati prema zbiru proizvodnje ovih elektrana. Rezolucija podataka je svakih 15 minuta za Krenicu1 i Hodovo1, odnosno 1 sat za Enelectronic, te će se izvršiti uzorkovanje na satnu rezoluciju. Tehnički podaci o FNE su dati na slikama:


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

## Priprema fajlova

In [ ]:
def open_the_file(filename,sheets):
    filex=pd.read_excel(filename,sheets,header=0)
    return filex #otvaranje

In [ ]:
def drop_empty_rows(filex):
    filex.drop(filex[filex[0]=='Diagram values'].index,inplace=True)
    filex.dropna(subset=[0],inplace=True)
    return filex #brisanje praznih redova

In [ ]:
#dodavanje vremenskog otiska
def assign_timestamp(filex,start,end,freq):
    filex=filex.assign(Timestamp=pd.date_range(start,end, freq=freq).values)
    return filex


In [ ]:
def set_index(filex):
    filex=filex.set_index(filex['Timestamp'].astype('datetime64[ns]'))
    return filex
#index

In [ ]:
from datetime import datetime
def assign_date(filex):
    filex=filex.assign(Date=filex.index.to_pydatetime())
    return filex

#dodjela datuma

In [ ]:
file1,file3=open_the_file("enelectronic_2019.xlsx",0),open_the_file("kr_hod_2019.xlsx",0)
file1_2020,file3_2020=open_the_file("enelectronic_2020.xlsx",0),open_the_file("kr_hod_2020.xlsx",0)
file1_2021,file3_2021=open_the_file("enelectronic_2021.xlsx",0),open_the_file("kr_hod_2021.xlsx",0)

In [ ]:
file1 = file1.drop(index=range(23))
file1=file1.assign(Timestamp=pd.date_range("2019-01-01 00:00:00","2020-01-01", freq="1H").values)
file3=file3.assign(Timestamp=pd.date_range("2019-01-01 00:15:00","2020-01-01",freq= "15T").values)

file1_2020 = file1_2020.drop(index=range(23))
file1_2020=file1_2020.assign(Timestamp=pd.date_range("2020-01-01 00:00:00","2020-12-31", freq="1H").values)
file3_2020=file3_2020.assign(Timestamp=pd.date_range("2020-01-01 00:15:00","2021-01-01",freq= "15T").values)


file1_2021 = file1_2021.drop(index=range(23))
file1_2021=file1_2021.assign(Timestamp=pd.date_range("2021-01-01 00:00:00","2022-01-01", freq="1H").values)
file3_2021=file3_2021.assign(Timestamp=pd.date_range("2021-01-01 00:15:00","2022-01-01",freq= "15T").values)

In [ ]:
new_columns_kr=['Unnamed: 0','Krenica1PowerMean values  [kW]','Hodovo1PowerMean values  [kW]']
new_columns_ene=['Unnamed: 0',"Enelectronic - Power, kW","Enelectronic - Total yield, kW"]


file1,file3=set_index(file1),set_index(file3)
file2,file4=file1.resample('1H').sum(),file3.resample('1H').sum()



file2.columns=new_columns_ene
file4.columns=new_columns_kr
file2,file4=assign_date(file2),assign_date(file4)
#file4.columns=["Hour","Enelectronic - Power, W","Enelectronic - Total yield, W","Timestamp","Date"]




file1_2020,file3_2020=set_index(file1_2020),set_index(file3_2020)
file2_2020,file4_2020=file1_2020.resample('1H').sum(),file3_2020.resample('1H').sum()

file2_2020.columns=new_columns_ene
file4_2020.columns=new_columns_kr
file2_2020,file4_2020=assign_date(file2_2020),assign_date(file4_2020)
#file4.columns=["Hour","Enelectronic - Power, W","Enelectronic - Total yield, W","Timestamp","Date"]


file1_2021,file3_2021=set_index(file1_2021),set_index(file3_2021)
file2_2021,file4_2021=file1_2021.resample('1H').sum(),file3_2021.resample('1H').sum()

file2_2021.columns=new_columns_ene
file4_2021.columns=new_columns_kr
file2_2021,file4_2021=assign_date(file2_2021),assign_date(file4_2021)
#file4.columns=["Hour","Enelectronic - Power, W","Enelectronic - Total yield, W","Timestamp","Date"]

Grafički prikaz će uključivati i analizu po mjesecima, te će se ta kolona uključiti u dataset.

In [ ]:

file2=file2.assign(Month=file2["Date"].dt.month)
file4=file4.assign(Month=file4["Date"].dt.month)


file2_2020=file2_2020.assign(Month=file2_2020["Date"].dt.month)
file4_2020=file4_2020.assign(Month=file4_2020["Date"].dt.month)


file2_2021=file2_2021.assign(Month=file2_2021["Date"].dt.month)
file4_2021=file4_2021.assign(Month=file4_2021["Date"].dt.month)

In [ ]:
file2.head(2)

In [ ]:
file4.head(2)

In [ ]:
file_total=pd.merge(file2,file4,how='inner', left_on=['Date',"Month"], right_on=['Date',"Month"])
file_total_2020=pd.merge(file2_2020,file4_2020,how='inner', left_on=['Date',"Month"], right_on=['Date',"Month"])
file_total_2021=pd.merge(file2_2021,file4_2021,how='inner', left_on=['Date',"Month"], right_on=['Date',"Month"])

In [ ]:
file_total.head(2)

In [ ]:
file_total.groupby("Month")[['Krenica1PowerMean values  [kW]','Hodovo1PowerMean values  [kW]','Enelectronic - Power, kW']].sum().plot(kind="bar")

In [ ]:
file_total_2020.groupby("Month")[['Krenica1PowerMean values  [kW]','Hodovo1PowerMean values  [kW]','Enelectronic - Power, kW']].sum().plot(kind="bar")

In [ ]:
file_total_2021.groupby("Month")[['Krenica1PowerMean values  [kW]','Hodovo1PowerMean values  [kW]','Enelectronic - Power, kW']].sum().plot(kind="bar")

In [ ]:
(file_total["Krenica1PowerMean values  [kW]"].sum(axis=0)+file_total["Hodovo1PowerMean values  [kW]"].sum()+file_total["Enelectronic - Power, kW"].sum())/1000000

In [ ]:
(file_total_2020["Krenica1PowerMean values  [kW]"].sum(axis=0)+file_total_2020["Hodovo1PowerMean values  [kW]"].sum()+file_total_2020["Enelectronic - Power, kW"].sum())/1000000

In [ ]:
(file_total_2021["Krenica1PowerMean values  [kW]"].sum(axis=0)+file_total_2021["Hodovo1PowerMean values  [kW]"].sum()+file_total_2021["Enelectronic - Power, kW"].sum())/1000000

In [ ]:
%pip install selenium

In [ ]:
# import re for reges
import re

import time
from bs4 import BeautifulSoup

# Import webdriver to initialise a browser
from selenium import webdriver
from selenium.webdriver import ActionChains

In [ ]:
# website that we want to scrap
url = "https://www.sunnyportal.com/Templates/PublicPageOverview.aspx?page=7b8e7dd9-4d7a-4539-9501-f2f36df09d61&plant=0b181aa3-03c7-446c-af6d-3ed6457a5394&splang=en-GB"

# Initialize webdriver and put the path where download the driver
driver = webdriver.Chrome('path_to_folder/chromedriver')

# Launch Chrome and pass the url
driver.get(url)